In [23]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from copy import deepcopy

from scipy import ndimage
from skimage.measure import label

from numpy import linalg
from scipy.sparse.linalg import eigs

from skimage.exposure import match_histograms

from utils import is_wing_facing_up, segment_contour

In [21]:
# initialize with the ones we couldn't find any lines in
seen = []
problem_wings = ['2024_06_05_h01bee23.png',
 '2024_06_18_h11b13.png',
 '2024_06_05_h01bee19.png',
 '2024_06_18_h11b11.png',
 '2024_06_20_h14b41.png',
 '2024_06_20_h14b03.png',
 '2024_07_23_h21b28.png',
 '2024_07_05_h21b29.png',
 '2024_06_18_h02b12.png',
 '2024_06_20_h14b42.png',
 '2024_07_31_h31b74.png',
 '2024_06_18_h11b17.png',
 '2024_06_20_h14b02.png',
 '2024_06_18_h11b19.png',
 '2024_07_31_h31b59.png',
 '2024_06_19_h13b37.png',
 '2024_06_20_h14b22.png',
 '2024_06_24_h12b18.png',
 '2024_06_19_h01b21.png',
 '2024_06_05_h01bee27.png',
 '2024_06_20_h14b40.png',
 '2024_06_05_h01bee46.png',
 '2024_06_19_h01b23.png',
 '2024_06_05_h01bee16.png',
 '2024_07_22_h33b17.png',
 '2024_06_24_h12b31.png',
 '2024_06_18_h11b04.png',
 '2024_06_13_h05b02.png',
 '2024_06_27_h14b18.png',
 '2024_06_26_h04b41.png',
 '2024_06_11_h01b40.png',
 '2024_07_04_h14b40.png']

In [24]:
seen = []
problem_wings = []
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import numpy as np
import os
from PIL import Image, ImageFilter

def imageEditor(folder, filename, blur, blurRadius): 
	# Opens a image in RGB mode
	im = Image.open('{}/{}'.format(folder, filename))

	if(blur == True):
		# blur only if checkbox checked
		im = im.filter(ImageFilter.GaussianBlur(blurRadius))
		
	# show the image
	plt.figure(figsize=(8,8))
	plt.imshow(im)
	plt.axis('off')
	plt.title(filename)
	print(filename)
	plt.show()

# creating the widgets
folderWidget = widgets.Text('../2_live_bees/2_card_segs_and_orig')
fns = [x for x in sorted(os.listdir(folderWidget.value)) if x not in seen]
fileWidget = widgets.Dropdown(options = fns)
# update function to update fileslist when folder name changes
def updateFilesList(*args):
	filesList = [file for file in os.listdir(folderWidget.value) if file.endswith(('.png', '.jpg', '.jpeg','.gif'))]
	fileWidget.options = filesList
	fileWidget.value = filesList[0]

# observer folder widget and call updateFilesList whenever its value changes
folderWidget.observe(updateFilesList, 'value')

interact(
	imageEditor,
	filename = fileWidget,
	folder = folderWidget,
	blur = widgets.Checkbox(value = False),
	blurRadius = widgets.FloatSlider(min = 0, max = 5, description = "Blur Radius")
)

button_prev = widgets.Button(description="Previous!")
button_next = widgets.Button(description="Next!")
button_add = widgets.Button(description="Add to list")
button_remove = widgets.Button(description="Remove from list")

output = widgets.Output()

display(button_prev)
display(button_next)
display(button_add)
display(button_remove, output)

def on_button_prev_clicked(a):
    with output:
        cur = fileWidget.get_state()['index']
        n_fps = len(fileWidget.get_state()['_options_labels'])
        if cur > 0:
            fileWidget.set_state({'index':cur - 1})
            print(f"{cur} / {n_fps}", end='\r')

def on_button_next_clicked(b):
    global seen
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        n_fps = len(fileWidget.get_state()['_options_labels'])
        fileWidget.set_state({'index':cur + 1})
        print(f"{cur} / {n_fps}", end='\r')

    if fn not in seen:
        seen += [fn]
    

def on_button_add_clicked(c):
    global problem_wings
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        print(f"Added {fn}", end='\r')

    if fn not in problem_wings:
        problem_wings += [fn]

def on_button_remove_clicked(c):
    global problem_wings
    with output:
        fns = fileWidget.get_state()['_options_labels']
        cur = fileWidget.get_state()['index']
        fn = fns[cur]
        print(f"Removed {fn}", end='\r')
    
    problem_wings.remove(fn)

button_prev.on_click(on_button_prev_clicked)
button_next.on_click(on_button_next_clicked)
button_add.on_click(on_button_add_clicked)
button_remove.on_click(on_button_remove_clicked)

interactive(children=(Text(value='../2_live_bees/2_card_segs_and_orig', description='folder'), Dropdown(descri…

Button(description='Previous!', style=ButtonStyle())

Button(description='Next!', style=ButtonStyle())

Button(description='Add to list', style=ButtonStyle())

Button(description='Remove from list', style=ButtonStyle())

Output()

In [19]:
problem_wings = []
seen = []